In [1]:
import torch
from torch import nn
import add_cuda

In [3]:
class AddFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input_left, input_right):
        outputs = add_cuda.forward(input_left, input_right)
        new_h = outputs[0]
        # variables = outputs[1:] + [weights]
        variables = [input_left, input_right]
        ctx.save_for_backward(*variables)
        return new_h

    @staticmethod
    def backward(ctx, grad_h):
        grad_input_left, grad_input_right = add_cuda.backward(
            grad_h.contiguous(), *ctx.saved_tensors
        )
        return grad_input_left, grad_input_right


class Add(torch.nn.Module):
    def __init__(self, size):
        super().__init__()
        # self.input_features = input_features
        # self.state_size = state_size
        # self.weights = torch.nn.Parameter(
        #     torch.empty(3 * state_size, input_features + state_size))
        # self.bias = torch.nn.Parameter(torch.empty(3 * state_size))
        # self.reset_parameters()
        self.input_left = nn.Parameter(torch.arange(size).float())
        self.input_right = nn.Parameter(torch.arange(size).float())

    def forward(self):
        return AddFunction.apply(self.input_left, self.input_right)
    
    # def forward(self):
    #     return self.input_left + self.input_right

In [4]:
module = Add(128).cuda()

In [5]:
module().sum().backward()


In [7]:
module.input_right.grad

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], device='cuda:0')